In [1]:
using Logging, LightGraphs, Revise
global_logger(ConsoleLogger(stdout, Logging.Info));

In [2]:
using PorousMaterials, MOFun

In [3]:
s_moty = moiety("find-replace/2-!-p-phenylene")
r_moty = moiety("2-acetylamido-p-phenylene")
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)
search = s_moty ∈ parent;

In [4]:
function config_idx_arr(configs::Array{Configuration}, search::Search)::Array{Int}
    c = Int[]
    if configs == Configuration[] # if configs not specified, choose one randomly
        push!(c, rand(1:search.num_isomorphisms))
    else # find indices of specified configs
        for config in configs
            for i in 1:search.num_isomorphisms
                if search.results[i].configuration == config
                    push!(c, i)
                    break
                end
            end
        end
    end
    return c
end

config_idx_arr (generic function with 1 method)

In [13]:
function accumulate_bonds!(bonds::Array{Tuple{Int,Int}}, s2p_isom::Array{Int}, parent::Crystal, m2r_isom::Array{Int}, r_moty::Crystal, xrms::Array{Crystal})
    # loop over s2p_isom
    for (s, p) in enumerate(s2p_isom)
        # find neighbors of parent_subset atoms
        n = LightGraphs.neighbors(parent.bonds, p)
        # loop over neighbors
        for nᵢ in n
            # if neighbor not in s2p_isom, must bond it to r_moty replacement of parent_subset atom
            if !(nᵢ ∈ s2p_isom)
                # ID the atom in r_moty
                r = m2r_isom[s]

                # add the index offset
                r += parent.atoms.n + length(xrms) * r_moty.atoms.n

                # push bond to array
                push!(bonds, (nᵢ, r))
            end
        end
    end
end

accumulate_bonds! (generic function with 2 methods)

In [14]:
function build_replacement_data(c::Array{Int}, search::Search, parent::Crystal, s_moty::Crystal, r_moty::Crystal, m2r_isom::Array{Int}, mask::Crystal
        )::Tuple{Array{Crystal},Array{Int},Array{Tuple{Int,Int}}}
    xrms = Crystal[]
    del_atoms = Int[]
    bonds = Tuple{Int,Int}[] # each tuple (i,j) encodes a parent[i] -> xrms[k][j] bond
    for cᵢ in c
        # find isomorphism
        s2p_isom = search.results[cᵢ].isomorphism
        # find parent subset
        parent_subset = deepcopy(parent[s2p_isom])
        # adjust coordinates for periodic boundaries
        MOFun.adjust_for_pb!(parent_subset)
        # record the center of xtal_subset so we can translate back later
        parent_subset_center = MOFun.geometric_center(parent_subset)
        # shift to align centers at origin
        MOFun.center_on_self!.([parent_subset, s_moty])
        # do orthog. Procrustes for s_moty-to-parent and mask-to-replacement alignments
        rot_s2p = MOFun.s2p_op(s_moty, parent_subset)
        rot_r2m = MOFun.r2m_op(r_moty, s_moty, m2r_isom, mask.atoms)
        # transform r_moty by rot_r2m, rot_s2p, and xtal_subset_center, align to parent
        # (this is now a crystal to add)
        xrm = MOFun.xform_r_moty(r_moty, rot_r2m, rot_s2p, parent_subset_center, parent)
        push!(xrms, xrm)
        # push obsolete atoms to array
        for x in s2p_isom
            push!(del_atoms, x) # this can create duplicates; remove them later
        end
        # clean up del_atoms
        del_atoms = unique(del_atoms)
        # accumulate bonds
        accumulate_bonds!(bonds, s2p_isom, parent, m2r_isom, r_moty, xrms)
    end
    return xrms, del_atoms, bonds
end

build_replacement_data (generic function with 1 method)

In [15]:
function fndrplc(s_moty::Crystal, r_moty::Crystal, parent::Crystal;
        configs::Array{Configuration}=Configuration[], 
        search::Search=Search(),
        kwargs...)::Crystal
    # mutation guard
    s_moty, r_moty = deepcopy.([s_moty, r_moty])
    # if search not specified, do it now
	search = MOFun.isequal(search, Search()) ? s_moty ∈ parent : search
    # if there are no replacements to be made, just return the parent
    if search.num_isomorphisms == 0
        return parent
    end
    # determine configuration index array
    c = config_idx_arr(configs, search)
    # determine s_mask (which atoms from s_moty are NOT in r_moty?)
    mask = s_moty[MOFun.idx_filter(s_moty, MOFun.r_group_indices(s_moty))]
    # get isomrphism between s_moty/mask and r_moty
    m2r_isom = (mask ∈ r_moty).results[1].isomorphism
    # shift all r_moty nodes according to center of isomorphic subset
    r_moty.atoms.coords.xf .-= MOFun.geometric_center(r_moty[m2r_isom])
    # loop over configs to build replacement data
    xrms, del_atoms, bonds = build_replacement_data(c, search, parent, s_moty, r_moty, m2r_isom, mask)
    # append temporary crystals to parent
    xtal = Crystal("TODO", parent.box, parent.atoms + sum([xrm.atoms for xrm in xrms]), Charges{Frac}(0))
    # create bonds from dictionary
    for (p, r) in bonds
        add_edge!(xtal.bonds, p, r)
    end
    # delete atoms from array and return result
    return xtal[[x for x in 1:xtal.atoms.n if !(x ∈ del_atoms)]]
end

# dispatch to handle single config w/o array-wrapping
function fndrplc(s, r, p; config, kwargs...)
    return fndrplc(s, r, p, configs=[config], kwargs...)
end

fndrplc (generic function with 2 methods)

In [16]:
xtal = find_replace(s_moty, r_moty, parent, search=search, configs=[Configuration(1,1), Configuration(2,1)])

Name: TODO
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 25.832000 Å. b = 25.832000 Å, c = 25.832000 Å
	Volume of unit cell: 17237.492730 Å³

	# atoms = 438
	# charges = 0
	chemical formula: Dict(:N => 1,:Zn => 16,:H => 51,:O => 53,:C => 98)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [17]:
write_cif(xtal, "OUTPUT")